In [79]:
import numpy as np
import scipy.misc
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [178]:
culled_data = pd.read_csv('culled_data.csv.gz', dtype={'line':'category', 'vehicle':'category', 'precipType':'category'})

In [181]:
culled_data['line'] = culled_data['line'].cat.codes
culled_data['vehicle'] = culled_data['vehicle'].cat.codes
culled_data['precipType'] = culled_data['precipType'].cat.codes

In [199]:
culled_data['latitude'].fillna(culled_data['latitude'].mean(), inplace=True)
culled_data['longitude'].fillna(culled_data['longitude'].mean(), inplace=True)
culled_data['visibility'].fillna(culled_data['visibility'].mean(), inplace=True)
culled_data['delay'].fillna(culled_data['delay'].mean(), inplace=True)

# Split into training and test data

In [255]:
targets = culled_data['late']
data_vect = culled_data.drop(['late'], axis=1)
data_vect = data_vect.drop(['delay'], axis=1)

targets = np.array(targets)
data_vect = np.array(data_vect)

In [256]:
training_data, test_data, train_target, test_target = train_test_split(data_vect, targets, train_size=0.8)
print('training_data size = ', len(training_data))
print('test_data size = ', len(test_data))

training_data size =  272578
test_data size =  68145


# Baseline (We have to beat this score)
This is the success rate if you always guess that the bus is late

In [258]:
dummy = DummyClassifier('most_frequent')
dummy.fit(training_data, train_target)
dummy.score(test_data, test_target)

0.75895516912466066

# Logistic Regression

In [257]:
model = LogisticRegression()
model.fit(training_data, train_target)
model.score(test_data, test_target)

0.75895516912466066